In [1]:
%cd ..
%pwd

/Users/areshytko/repos/productivity-dashboard


'/Users/areshytko/repos/productivity-dashboard'

## Load data

In [2]:
from dashboard.load import load
import dashboard.config as config 
from dashboard.gheets import ManualFlow

import json

/Users/areshytko/repos/productivity-dashboard/.venv/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
flow = ManualFlow()
url = flow.get_url()
with open('notebooks/url.json', 'w') as wf:
    json.dump(url, wf)

In [4]:
flow.put_code("4/1AY0e-g5XNIzd3u-AMkZxqjo4XaitG6m2vrizO2od2U8FCi6pOy6sZ7TI4Fo")

In [4]:
creds = flow.get_google_token()

In [5]:
data = load(
    credentials=creds,
    pomodoros_spreadsheet_id=config.POMODOROS_SPREADSHEET_ID,
    pomodoros_range=config.POMODOROS_RANGE,
    activities_spreadsheet_id=config.ACTIVITIES_SPREADSHEET_ID,
    activities_range=config.ACTIVITIES_RANGE
)

In [6]:
data.df.head()

,Week,Date,Activity,Comment,Pomodoros,Planned,learn,do,career,green,red,personal,hobby,society,life,health
0,2,2021-01-06,Learn/System Design,None,2.0,4.0,True,False,True,True,False,False,False,False,False,False
1,2,2021-01-06,Learn/Golang,None,2.0,2.0,True,False,True,True,False,False,False,False,False,False
2,2,2021-01-06,Learn/Graph Algorithms,None,2.0,2.0,True,False,False,True,False,False,True,False,False,False
3,2,2021-01-06,Learn/Personal Development,None,1.0,1.0,True,False,False,False,False,True,False,False,False,False
4,2,2021-01-06,Do/Algorithms,None,NaN,2.0,False,True,True,True,False,False,False,False,False,False


## Process data

In [8]:
from dashboard.process import compute_weekly_stats

weekly_stats = compute_weekly_stats(data)

weekly_stats.df.head()

,Week,from_date,to_date,done,planned,avg_done,avg_planned,do_learn_ratio,red_green_ratio,balance_coef,complete_rate
0,2,2021-01-06,2021-01-10,17.0,57.0,2.428571,8.142857,0.117647,0.000000,0.384001,0.298246
1,3,2021-01-11,2021-01-17,100.0,120.0,14.285714,17.142857,0.939394,0.010417,0.331964,0.833333
2,4,2021-01-18,2021-01-24,20.0,58.0,2.857143,8.285714,0.000000,0.000000,0.158114,0.344828
3,5,2021-01-25,2021-01-31,39.0,58.0,5.571429,8.285714,0.162162,0.058824,0.181010,0.672414
4,6,2021-02-01,2021-02-14,69.0,106.0,9.857143,15.142857,0.462963,0.193548,0.256130,0.650943


## Get current week

In [10]:
x = weekly_stats.df.loc[weekly_stats.df.Week == weekly_stats.df.Week.max(), :]

In [26]:
import datetime
today = datetime.date.today()

x.from_date.iloc[0] <= today <= x.to_date.iloc[0]

True

In [27]:
x

,Week,from_date,to_date,done,planned,avg_done,avg_planned,do_learn_ratio,red_green_ratio,balance_coef,complete_rate
10,14,2021-04-05,2021-04-11,11.0,0.0,1.571429,0.0,0.636364,0.0,0.883901,inf


## Get sliding window

In [32]:
current = x.Week.iloc[0]

weekly_stats.df.loc[ (current - weekly_stats.df.Week <= 3) & (current - weekly_stats.df.Week > 0), :]

,Week,from_date,to_date,done,planned,avg_done,avg_planned,do_learn_ratio,red_green_ratio,balance_coef,complete_rate
7,11,2021-03-15,2021-03-21,27.0,39.0,3.857143,5.571429,0.481481,0.000000,0.433847,0.692308
8,12,2021-03-22,2021-03-28,23.0,0.0,3.285714,0.000000,0.300000,0.722222,0.304705,inf
9,13,2021-03-29,2021-04-04,8.0,0.0,1.142857,0.000000,1.000000,0.375000,0.539676,inf


## Get zone

In [35]:
today

datetime.date(2021, 4, 6)

In [34]:
today.weekday()

1

In [36]:
datetime.date(2021, 4, 5).weekday()

0

In [37]:
7 - today.weekday()

6

## Suggested action

In [51]:
data.df.loc[data.df.Date == pd.to_datetime(today), :]

,Week,Date,Activity,Comment,Pomodoros,Planned,learn,do,career,green,red,personal,hobby,society,life,health
402,14,2021-04-06,Do/Work,None,NaN,NaN,False,False,True,False,True,False,False,False,False,False
403,14,2021-04-06,Learn/Medium Articles,"узнать по визе, договориться по поводу отпуска...",NaN,NaN,True,False,False,True,False,False,False,False,False,False
404,14,2021-04-06,Learn/Data Engineering,Spark course,NaN,NaN,True,False,True,True,False,False,False,False,False,False
405,14,2021-04-06,Personal Dashboard,None,1.0,NaN,False,True,False,True,False,False,True,False,False,False
406,14,2021-04-06,Learn/How to Write,None,NaN,NaN,True,False,False,True,False,True,False,True,False,False


In [40]:
today

datetime.date(2021, 4, 6)

In [49]:
import pandas as pd
pd.to_datetime(today)

Timestamp('2021-04-06 00:00:00')

In [44]:
data.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Week       431 non-null    int16         
 1   Date       431 non-null    datetime64[ns]
 2   Activity   431 non-null    object        
 3   Comment    106 non-null    object        
 4   Pomodoros  119 non-null    float64       
 5   Planned    229 non-null    float64       
 6   learn      431 non-null    bool          
 7   do         431 non-null    bool          
 8   career     431 non-null    bool          
 9   green      431 non-null    bool          
 10  red        431 non-null    bool          
 11  personal   431 non-null    bool          
 12  hobby      431 non-null    bool          
 13  society    431 non-null    bool          
 14  life       431 non-null    bool          
 15  health     431 non-null    bool          
dtypes: bool(10), datetime64[ns](1), float64(2), 

In [59]:
weekly_stats.df.loc[(weekly_stats.df.Week > 3) & (weekly_stats.df.Week < 14),['Week', 'done']].median()

Week     8.5
done    24.0
dtype: float64